<a href="https://colab.research.google.com/github/apicem7217/Clase-12/blob/Phyton/Ch24_CodeOptimization_ExecutionTime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimización de código y tiempo de ejecución
<a href="https://colab.research.google.com/github/rambasnet/FDSPython-Notebooks/blob/master/Ch20-CodeOptimization-ExecutionTime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## encontrar el tiempo de ejecución real del código - timeit
- timeit - mide el tiempo de ejecución de pequeños fragmentos de código
- timeit.timeit(stmt='pass', setup='pass', timer=[default timer], number=1000000, globals=None)
- returns time in seconds

In [ ]:
import time
print(time.time()) # 1609954317.943479

1690336037.0357141


In [ ]:
import time
inicio = time.time()

# Código a medir
time.sleep(1)
# -------------

fin = time.time()
print(fin-inicio) # 1.0005340576171875

1.0050146579742432


Por otro lado, también podemos crear un decorador que use time, lo que nos permitirá medir el tiempo de ejecución de nuestras funciones sin repetir tanto código. Creamos el decorador de la siguiente manera:

In [ ]:
def mide_tiempo(funcion):
    def funcion_medida(*args, **kwargs):
        inicio = time.time()
        c = funcion(*args, **kwargs)
        print(time.time() - inicio)
        return "time:",c
    return funcion_medida

Ahora podemos usar el decorador mide_tiempo con @ antes de nuestra función, y cada vez que la llamemos se imprimirá por pantalla el tiempo que tardó en ejecutarse.

In [ ]:
@mide_tiempo
def calcula_pares(n):
    return [i for i in range(n) if i%2 == 0]

calcula_pares(5)


5.7220458984375e-06


('time:', [0, 2, 4])

Cuando tenemos diferentes bloques de código una forma es hacerlo de esta manera, estableciendo dos tiempos, inicial y final y restando el tiempo

In [ ]:
import time
t1 = time.perf_counter()

### Your code goes here ###

t2 = time.perf_counter()
print('time taken to run:',t2-t1)

time taken to run: 3.934899996238528e-05


# Fragmentos de código de tiempo: %timeit y %time

En el proceso de desarrollo de código y creación de canalizaciones de procesamiento de datos, a menudo hay compensaciones que puede hacer entre varias implementaciones.
Al principio del desarrollo de su algoritmo, puede ser contraproducente preocuparse por esas cosas. Como dijo en broma Donald Knuth: "Deberíamos olvidarnos de las pequeñas eficiencias, digamos alrededor del 97% del tiempo: la optimización prematura es la raíz de todos los males".

Pero una vez que tenga su código funcionando, puede ser útil profundizar un poco en su eficiencia.
A veces es útil verificar el tiempo de ejecución de un comando dado o un conjunto de comandos; otras veces es útil examinar un proceso de varias líneas y determinar dónde se encuentra el cuello de botella en alguna serie complicada de operaciones.
IPython brinda acceso a una amplia gama de funcionalidades para este tipo de sincronización y creación de perfiles de código.
Aquí discutiremos los siguientes comandos mágicos de IPython:

- `%time`: cronometrar la ejecución de una única sentencia
- `%timeit`: tiempo de ejecución repetida de una sola declaración para mayor precisión
- `%prun`: Ejecutar código con el generador de perfiles
- `%lprun`: Ejecutar código con el generador de perfiles línea por línea
- `%memit`: Mide el uso de memoria de una sola declaración
- `%mprun`: ejecuta el código con el generador de perfiles de memoria línea por línea

Los últimos cuatro comandos no se incluyen con IPython; para usarlos necesitará obtener las extensiones `line_profiler` y `memory_profiler`, que discutiremos en las siguientes secciones.

In [ ]:
%timeit sum(range(100))

1.14 µs ± 379 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [ ]:
%%timeit
total = 0
for i in range(1000):
    for j in range(1000):
        total += i * (-1) ** j

381 ms ± 8.55 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
import random
L = [random.random() for i in range(100000)]
%timeit L.sort()

795 µs ± 144 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
import random
L = [random.random() for i in range(100000)]
print("sorting an unsorted list:")
%time L.sort()

sorting an unsorted list:
CPU times: user 29 ms, sys: 0 ns, total: 29 ms
Wall time: 29 ms


In [ ]:
print("sorting an already sorted list:")
%time L.sort()

sorting an already sorted list:
CPU times: user 3.69 ms, sys: 0 ns, total: 3.69 ms
Wall time: 3.63 ms


In [ ]:
%%time
total = 0
for i in range(1000):
    for j in range(1000):
        total += i * (-1) ** j

CPU times: user 804 ms, sys: 0 ns, total: 804 ms
Wall time: 825 ms


## Creación de perfiles de scripts completos: %prun

Un programa se compone de muchas sentencias individuales y, a veces, cronometrar estas sentencias en contexto es más importante que cronometrarlas por sí solas.
Python contiene un generador de perfiles de código incorporado (sobre el que puede leer en la documentación de Python), pero IPython ofrece una forma mucho más conveniente de usar este generador de perfiles, en la forma de la función mágica `%prun`.

A modo de ejemplo, definiremos una función simple que hace algunos cálculos:

In [ ]:
def sum_of_lists(N):
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
    return total

In [ ]:
%prun sum_of_lists(1000000)

El resultado es una tabla que indica, en orden de tiempo total en cada llamada de función, dónde está pasando la mayor parte del tiempo la ejecución. En este caso, la mayor parte del tiempo de ejecución está en la comprensión de la lista dentro de `sum_of_lists`.
A partir de aquí, podríamos empezar a pensar en qué cambios podríamos hacer para mejorar el rendimiento del algoritmo.

Para obtener más información sobre `%prun`, así como sus opciones disponibles, utilice la función de ayuda de IPython (es decir, escriba `%prun?` en el indicador de IPython).

# Optimización de código Python
- https://wiki.python.org/moin/PythonSpeed/PerformanceTips

## Resumen

- use funciones y variables locales en lugar de globales, aunque las llamadas a funciones son relativamente altas
- evitar el punto. especificador de acceso a miembros
- list.sort integrado por el usuario y ordenado con clave usando la función itemgetter si es necesario
- evitar la concatenación de cadenas; use "".join(alguna lista) en su lugar
- use la comprensión de listas y map() en lugar de bucles
- mientras trabaja con dict (especialmente inicializando), use try catch en lugar de key test
- usa la clase defaultdict de las colecciones
- hacer cosas con menos frecuencia - cambiar sys.setswitchinterval(sys.maxsize) al máximo posible si no se ejecutan subprocesos ni se captan señales
- usar sumas y restas en lugar de multiplicaciones y divisiones
- evitar la recursividad; especialmente si no puede aumentar el límite de recurrencia del sistema
    - sys.setrecursionlimit(10**6)

In [ ]:
import sys
sys.getswitchinterval()

0.005

In [ ]:
# cuál es el tamaño máximo del sistema
sys.maxsize

9223372036854775807

In [ ]:
sys.setswitchinterval(100000)

In [ ]:
sys.getswitchinterval()

100000.0

In [ ]:
import timeit

In [ ]:
code = '''
x = 47
x = x * 2
'''
timeit.timeit(stmt=code) # runs 1M times

0.05023645599999327

In [ ]:
code = '''
x = 47
x = x << 1
'''
timeit.timeit(stmt=code) # runs 1M times

0.0737487889999926

In [ ]:
code = '''
# x is local variable
x = 47
x = x + x
'''
timeit.timeit(stmt=code) # runs 1M times

0.10674129400000254

In [ ]:
# x is global variable
x = 47
code = '''
global x
x = x + x
'''
timeit.timeit(stmt=code, globals=globals()) # runs 1M times; takes ~12 seconds

16.425606916000007

In [ ]:

# imprimiendo algún resultado n veces
# la entrada y la salida son los principales cuellos de botella en el tiempo de ejecución
code = '''
import sys
n = 10000 #100K takes 5.x seconds on my macbook pro 2019
for i in range(n):
    print(i**2)
'''
timeit.timeit(stmt=code, number=1)

Se truncaron las últimas líneas 5000 del resultado de transmisión.
25000000
25010001
25020004
25030009
25040016
25050025
25060036
25070049
25080064
25090081
25100100
25110121
25120144
25130169
25140196
25150225
25160256
25170289
25180324
25190361
25200400
25210441
25220484
25230529
25240576
25250625
25260676
25270729
25280784
25290841
25300900
25310961
25321024
25331089
25341156
25351225
25361296
25371369
25381444
25391521
25401600
25411681
25421764
25431849
25441936
25452025
25462116
25472209
25482304
25492401
25502500
25512601
25522704
25532809
25542916
25553025
25563136
25573249
25583364
25593481
25603600
25613721
25623844
25633969
25644096
25654225
25664356
25674489
25684624
25694761
25704900
25715041
25725184
25735329
25745476
25755625
25765776
25775929
25786084
25796241
25806400
25816561
25826724
25836889
25847056
25857225
25867396
25877569
25887744
25897921
25908100
25918281
25928464
25938649
25948836
25959025
25969216
25979409
25989604
25999801
26010000
26020201
26030404
260406

3.2974181479999913

In [ ]:

# concatenar el resultado en una cadena e imprimir una vez
code = '''
n = 100000 # 100K takes 0.056 seconds on my 2019 macbook pro
ans = ''
for i in range(n):
    ans += str(i**2)
print(ans)
'''
timeit.timeit(stmt=code, number=1)

...


0.1097671269999978

In [ ]:
#concatenar el resultado en una cadena e imprimir una vez
code = '''
n = 10000000 # 100K takes 0.050 seconds on my 2019 macbook pro
ans = ''
for i in range(n):
    ans += str(i**2)
'''
timeit.timeit(stmt=code, number=1)

6.653653765000058

In [ ]:
# usar la lista para recopilar respuestas e imprimir una vez como cadena
code = '''
n = 10000000 # 100K takes 0.055 seconds on my 2019 macbook pro
ans = []
for i in range(n):
    ans.append(str(i**2)) #converting and appending string to list
ans = ''.join(ans) #typically print before storing into a variable
'''
timeit.timeit(stmt=code, number=1)

7.62956052800007

In [ ]:
# usar la lista para recopilar respuestas int e imprimir una vez como cadena
# no hay mucha diferencia en comparación con la recolección de cadenas en sí
code = '''
n = 10000000 # 100K takes 0.055 seconds on my 2019 macbook pro
ans = []
for i in range(n):
    ans.append(i**2) #appending int to list
#map to string and join to print
ans = ''.join(map(str, ans))
'''
timeit.timeit(stmt=code, number=1)

6.235775977999992

In [ ]:
def factorial_recurse(n):
    if(n == 0):
        return 1
    return n * factorial_recurse(n - 1)

In [ ]:
sys.getrecursionlimit()

1000000

In [ ]:
factorial_recurse(3000)

4149359603437854085556867093086612170951119194931809917689467657697558565123531950086000765217800342007518463538361711849575087111404590779455340216106833961162103790419917752206266339017968280516471969749596884245772876609710300372611109534024112711883315773881532843892973761302110631293037440148537872544607961029042949104979388812076251162513291700464166896211759020357517548898065357786891528509378246999467469919083209351106836382428706352226854433921377515048858810403681880909929291249714190050893899440471535147315453158744150996017426787508746036797411707236874727714398892068369161850360819845971809378445352395850537761108651116236314592088610855745087451394530543621371189815084719209442637420327502999633378494401477567141468082420749991471487835966972063895467058996017856948026338876711287106800495082740071712481947638640136919354435412031278660143479254995914353012065310340662550323102073835150219510314867361233873939509655146215934901578994994407231100442692483814014145548787273

In [ ]:
sys.setrecursionlimit(10**6)

In [ ]:
factorial_recurse(3000)

4149359603437854085556867093086612170951119194931809917689467657697558565123531950086000765217800342007518463538361711849575087111404590779455340216106833961162103790419917752206266339017968280516471969749596884245772876609710300372611109534024112711883315773881532843892973761302110631293037440148537872544607961029042949104979388812076251162513291700464166896211759020357517548898065357786891528509378246999467469919083209351106836382428706352226854433921377515048858810403681880909929291249714190050893899440471535147315453158744150996017426787508746036797411707236874727714398892068369161850360819845971809378445352395850537761108651116236314592088610855745087451394530543621371189815084719209442637420327502999633378494401477567141468082420749991471487835966972063895467058996017856948026338876711287106800495082740071712481947638640136919354435412031278660143479254995914353012065310340662550323102073835150219510314867361233873939509655146215934901578994994407231100442692483814014145548787273

In [ ]:
timeit.timeit('factorial_recurse(3000)', number=1, globals=globals())

0.005851690000099552

In [ ]:
def factorial_iter(n):
    fact = 1
    for i in range(1, n+1):
        fact *= i
    return fact

In [ ]:
factorial_iter(3000)

4149359603437854085556867093086612170951119194931809917689467657697558565123531950086000765217800342007518463538361711849575087111404590779455340216106833961162103790419917752206266339017968280516471969749596884245772876609710300372611109534024112711883315773881532843892973761302110631293037440148537872544607961029042949104979388812076251162513291700464166896211759020357517548898065357786891528509378246999467469919083209351106836382428706352226854433921377515048858810403681880909929291249714190050893899440471535147315453158744150996017426787508746036797411707236874727714398892068369161850360819845971809378445352395850537761108651116236314592088610855745087451394530543621371189815084719209442637420327502999633378494401477567141468082420749991471487835966972063895467058996017856948026338876711287106800495082740071712481947638640136919354435412031278660143479254995914353012065310340662550323102073835150219510314867361233873939509655146215934901578994994407231100442692483814014145548787273

In [ ]:
timeit.timeit('factorial_iter(3000)', number=1, globals=globals())

0.003503843000089546

## Time complexity
- Time complexity of various Python built-in data sturctures and functions
- https://wiki.python.org/moin/TimeComplexity

# Generación de perfiles línea por línea con %lprun
La generación de perfiles función por función de %prun es útil, pero a veces es más conveniente tener un informe de perfil línea por línea. Esto no está integrado en Python o IPython, pero hay un paquete line_profiler disponible para la instalación que puede hacer esto. Comience usando la herramienta de empaquetado de Python, pip, para instalar el paquete line_profiler:

In [ ]:
pip install line_profiler

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.9/661.9 kB 8.4 MB/s eta 0:00:00


Ahora el comando %lprun creará un perfil línea por línea de cualquier función. En este caso, debemos decirle explícitamente qué funciones nos interesa perfilar:

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f sum_of_lists sum_of_lists(5000)

La información en la parte superior nos da la clave para leer los resultados: el tiempo se informa en microsegundos y podemos ver dónde el programa está pasando la mayor parte del tiempo. En este punto, es posible que podamos usar esta información para modificar aspectos del script y hacer que funcione mejor para nuestro caso de uso deseado.

Para obtener más información sobre %lprun, así como sus opciones disponibles, utilice la función de ayuda de IPython (es decir, escriba %lprun? en el indicador de IPython).

## Uso de memoria de generación de perfiles: %memit y %mprun

Otro aspecto de la creación de perfiles es la cantidad de memoria que utiliza una operación.
Esto se puede evaluar con otra extensión de IPython, `memory_profiler`.
Al igual que con `line_profiler`, empezamos por `pip`-instalando la extensión:

```
pip install memory_profiler
```

Luego podemos usar IPython para cargarlo:

In [ ]:
pip install memory_profiler

In [ ]:
%load_ext memory_profiler

La extensión del generador de perfiles de memoria contiene dos funciones mágicas útiles: %memit (que ofrece un equivalente de medición de memoria de %timeit) y %mprun (que ofrece un equivalente de medición de memoria de %lprun). La función mágica %memit se puede usar de manera bastante simple:

In [ ]:
%memit sum_of_lists(1000000)

peak memory: 382.95 MiB, increment: 70.33 MiB


Vemos que esta función utiliza unos 382 MB de memoria.

Para una descripción línea por línea del uso de la memoria, podemos usar la función mágica `%mprun`.
Desafortunadamente, esto funciona solo para funciones definidas en módulos separados en lugar del cuaderno en sí, por lo que comenzaremos usando la celda mágica `%%file` para crear un módulo simple llamado `mprun_demo.py`, que contiene nuestra `sum_of_lists` función, con una adición que hará que nuestros resultados de perfiles de memoria sean más claros:

In [ ]:
%%file mprun_demo.py
def sum_of_lists(N):
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
        del L # remove reference to L
    return total

Writing mprun_demo.py


Ahora podemos importar la nueva versión de esta función y ejecutar el generador de perfiles de línea de memoria:

In [ ]:
from mprun_demo import sum_of_lists
%mprun -f sum_of_lists sum_of_lists(10)

Aquí, la columna Incremento nos dice cuánto afecta cada línea al presupuesto total de memoria: observe que cuando creamos y eliminamos la lista L, estamos agregando alrededor de 30 MB de uso de memoria. Esto se suma al uso de la memoria de fondo del propio intérprete de Python.

Para obtener más información sobre %memit y %mprun, así como sus opciones disponibles, utilice la función de ayuda de IPython (por ejemplo, escriba %memit? en el indicador de IPython).